In [53]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?item ?itemLabel ?country ?countryLabel ?alternative
WHERE 
{
  ?item wdt:P361 wd:Q83741704.
  ?item wdt:P17 ?country.
  OPTIONAL { ?country skos:altLabel ?alternative . FILTER (lang(?alternative) = "en") }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""


def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

In [65]:
import pandas as pd

dictionary = pd.json_normalize(results["results"]["bindings"])

In [66]:
dictionary = dictionary[['item.value','itemLabel.value',"country.value", "countryLabel.value", "alternative.value"]]

In [67]:
dictionary.columns = ["outbreak_qid", "outbreak", "country_qid", "country", "other_names"]

In [68]:
outbreak_qids = dictionary["outbreak_qid"]

dictionary["outbreak_qid"] = [item.split("/")[4] for item in outbreak_qids]
    
country_qids = dictionary["country_qid"]

dictionary["country_qid"] = [item.split("/")[4] for item in country_qids]    

In [71]:
dictionary.to_csv("country_to_outbreak_dictionary.csv")

In [72]:
dictionary

,outbreak_qid,outbreak,country_qid,country,other_names
0,Q83872291,2019–20 coronavirus outbreak in Japan,Q17,Japan,Nihon
1,Q83872291,2019–20 coronavirus outbreak in Japan,Q17,Japan,Nihon-koku
2,Q83872291,2019–20 coronavirus outbreak in Japan,Q17,Japan,Nippon-koku
3,Q83872291,2019–20 coronavirus outbreak in Japan,Q17,Japan,State of Japan
4,Q87186365,2020 coronavirus outbreak in the Republic of I...,Q27,Ireland,Southern Ireland
...,...,...,...,...,...
954,Q87709760,2020 coronavirus pandemic in Curaçao,Q25279,Curaçao,Kòrsou
955,Q87199320,2020 coronavirus outbreak in Palestine,Q219060,State of Palestine,Palestine
956,Q87199320,2020 coronavirus outbreak in Palestine,Q219060,State of Palestine,PA
957,Q87199320,2020 coronavirus outbreak in Palestine,Q219060,State of Palestine,Palestinian Authority
